In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext
from pyspark import HiveContext
from pyspark.sql.types import *
from pyspark.sql import Row, functions as F
#from pyspark.sql import col
from pyspark.sql.window import Window
#from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql.functions import lpad, substring
from pyspark.sql.functions import *
import os
import sys
import datetime

In [2]:
sc = SparkContext("yarn-client", "Citi_Yarn")
from pyspark.sql import HiveContext
sqlContext = HiveContext(sc)
hiveContext = HiveContext(sc)

In [3]:
catbeer_sas = sqlContext.read.format("com.github.saurfang.sas.spark").load("/user/cloudera/sas/cat_beer.sas7bdat")

In [4]:
catbeer_sas.show()

+---------------+---------+---------+----------+
|SponsorLastName|Continent|   Buyers|Price_unit|
+---------------+---------+---------+----------+
|        Seliger|  America|8456677.0|     45.69|
|        Goldman|   Europa| 322225.0|     51.71|
|        Guillen|     Asia| 588998.0|     14.66|
|          Geren|   Africa|   2664.0|       6.3|
+---------------+---------+---------+----------+



In [5]:
beer_sas = sqlContext.read.format("com.github.saurfang.sas.spark").load("/user/cloudera/sas/beer_sponsors.sas7bdat")

In [6]:
beer_sas.show(10)

+---------------+----------------+---------------+----------------+--------------------+---------------+-----------------+-------------+
|SponsorLastName|SponsorFirstName|ContribLastName|ContribFirstName| ContribOrganization|ContribEmployer|ContribOccupation|ContribAmount|
+---------------+----------------+---------------+----------------+--------------------+---------------+-----------------+-------------+
|          Geren|      Charles L.|           NULL|            NULL|             A&M PAC|           NULL|             NULL|       7500.0|
|          Geren|      Charles L.|           NULL|            NULL|Abbott Laboratori...|           NULL|             NULL|       2000.0|
|          Geren|      Charles L.|           NULL|            NULL|          Abbvie PAC|           NULL|             NULL|       2000.0|
|          Geren|      Charles L.|           NULL|            NULL|             ABC PAC|           NULL|             NULL|       3000.0|
|          Geren|      Charles L.|       

In [7]:
cruce_sas = beer_sas.alias("A").join(catbeer_sas.alias("B"), beer_sas.SponsorLastName == catbeer_sas.SponsorLastName,how='left').select("A.*","B.Continent","B.Price_Unit")

In [8]:
cruce_sas.printSchema()

root
 |-- SponsorLastName: string (nullable = true)
 |-- SponsorFirstName: string (nullable = true)
 |-- ContribLastName: string (nullable = true)
 |-- ContribFirstName: string (nullable = true)
 |-- ContribOrganization: string (nullable = true)
 |-- ContribEmployer: string (nullable = true)
 |-- ContribOccupation: string (nullable = true)
 |-- ContribAmount: double (nullable = true)
 |-- Continent: string (nullable = true)
 |-- Price_Unit: double (nullable = true)



In [70]:
# Agregamos columnas al dataframe que contiene el left join de los dataframes previos
# Agregamos diferentes tipos de columnas y funciones de Spark SQL 

cruce_columnas = cruce_sas.withColumn("Pais", lit("Mexico")).\
withColumn("Lote", lit(8989898 + 100)).\
withColumn("Precio", col("Price_Unit") * 10020.23 ).\
withColumn("Inicial", substring(col("Pais"),1,1)).\
withColumn("fecha_proceso", current_date()).\
withColumn("fecha_procesada", date_add(current_date(),-1)).\
withColumn("fecha_auditoria", current_timestamp()).\
withColumn("ejemplo_lpad", lpad(lit("77"),5,"0") ).\
withColumn("ejemplo_rpad", rpad(lit("77"),5,"0") ).\
withColumn("cast_string", col("ContribAmount").cast("string")).\
withColumn("cast_decimal",col("Price_Unit").cast("decimal(29,4)") )


In [73]:
#Imprimimos la estructura de dataframe
cruce_columnas.printSchema()

root
 |-- SponsorLastName: string (nullable = true)
 |-- SponsorFirstName: string (nullable = true)
 |-- ContribLastName: string (nullable = true)
 |-- ContribFirstName: string (nullable = true)
 |-- ContribOrganization: string (nullable = true)
 |-- ContribEmployer: string (nullable = true)
 |-- ContribOccupation: string (nullable = true)
 |-- ContribAmount: double (nullable = true)
 |-- Continent: string (nullable = true)
 |-- Price_Unit: double (nullable = true)
 |-- Pais: string (nullable = false)
 |-- Lote: integer (nullable = false)
 |-- Precio: double (nullable = true)
 |-- Inicial: string (nullable = false)
 |-- fecha_proceso: date (nullable = false)
 |-- fecha_procesada: date (nullable = false)
 |-- fecha_auditoria: timestamp (nullable = false)
 |-- ejemplo_lpad: string (nullable = false)
 |-- ejemplo_rpad: string (nullable = false)
 |-- cast_string: string (nullable = true)
 |-- cast_decimal: decimal(29,4) (nullable = true)



In [71]:
cruce_columnas.select("Lote","Precio","Inicial","fecha_proceso","fecha_procesada","fecha_auditoria","ejemplo_lpad","ejemplo_rpad","cast_string").show(10,False)

+-------+------------------+-------+-------------+---------------+-----------------------+------------+------------+-----------+
|Lote   |Precio            |Inicial|fecha_proceso|fecha_procesada|fecha_auditoria        |ejemplo_lpad|ejemplo_rpad|cast_string|
+-------+------------------+-------+-------------+---------------+-----------------------+------------+------------+-----------+
|8989998|457824.30869999994|M      |2021-01-14   |2021-01-13     |2021-01-14 18:35:53.071|00077       |77000       |5000.0     |
|8989998|457824.30869999994|M      |2021-01-14   |2021-01-13     |2021-01-14 18:35:53.071|00077       |77000       |1000.0     |
|8989998|457824.30869999994|M      |2021-01-14   |2021-01-13     |2021-01-14 18:35:53.071|00077       |77000       |2000.0     |
|8989998|457824.30869999994|M      |2021-01-14   |2021-01-13     |2021-01-14 18:35:53.071|00077       |77000       |1000.0     |
|8989998|457824.30869999994|M      |2021-01-14   |2021-01-13     |2021-01-14 18:35:53.071|00077  

In [72]:
cruce_columnas.printSchema()

root
 |-- SponsorLastName: string (nullable = true)
 |-- SponsorFirstName: string (nullable = true)
 |-- ContribLastName: string (nullable = true)
 |-- ContribFirstName: string (nullable = true)
 |-- ContribOrganization: string (nullable = true)
 |-- ContribEmployer: string (nullable = true)
 |-- ContribOccupation: string (nullable = true)
 |-- ContribAmount: double (nullable = true)
 |-- Continent: string (nullable = true)
 |-- Price_Unit: double (nullable = true)
 |-- Pais: string (nullable = false)
 |-- Lote: integer (nullable = false)
 |-- Precio: double (nullable = true)
 |-- Inicial: string (nullable = false)
 |-- fecha_proceso: date (nullable = false)
 |-- fecha_procesada: date (nullable = false)
 |-- fecha_auditoria: timestamp (nullable = false)
 |-- ejemplo_lpad: string (nullable = false)
 |-- ejemplo_rpad: string (nullable = false)
 |-- cast_string: string (nullable = true)
 |-- cast_decimal: decimal(29,4) (nullable = true)



In [26]:
#Funcion 1
def actualizaClave(clave):
    actualiza=(clave*10) + 20
    return actualiza

#Funcion que volvemos UDF
actualizaClave = udf(actualizaClave)


#Funcion 2
def calculo(z):
    return z * z
    
#Funcion que volvemos UDF
cuadrado = udf(calculo)

#Funcion 3
def mayusculas(z):
    return z.upper()

#Funcion que volvemos UDF
mayus = udf(mayusculas)

fecha=datetime.datetime.now().strftime ("%y%m%d")  


estado="CDMX"
pais="Mexico"
empresa="Apple"
anio="2018"

In [30]:
employees_modificado = employees_avr.
withColumn("CVE_DEPTO", concat(lit(estado),lit(","),lit(pais))).
withColumn("FCH_Proceso", lit(fecha).cast(StringType()))          
otro = employees_mysql.withColumn("Nva_Clave", cuadrado("emp_no")).
withColumn("Empresa", mayus(lit(empresa))).
withColumn("Anio_Reproceso",lit(anio))



In [31]:
employees_modificado.printSchema()

root
 |-- emp_no: integer (nullable = true)
 |-- birth_date: long (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- hire_date: long (nullable = true)
 |-- CVE_DEPTO: string (nullable = false)
 |-- FCH_Proceso: string (nullable = false)



In [9]:
#Guardar el resultado

otro.saveAsTable("spark_data.pyspark_udf",mode='overwrite')

root
 |-- emp_no: integer (nullable = true)
 |-- birth_date: long (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- hire_date: long (nullable = true)



In [32]:
dataframe_mysql = sqlContext.read.format("jdbc").options( url="jdbc:mysql://localhost:3306/employees",driver = "com.mysql.jdbc.Driver",dbtable = "employees",user="root", password="cloudera").load()

In [33]:
dataframe_mysql.where("emp_no > 13000").show()

+------+----------+-----------+---------+------+----------+
|emp_no|birth_date| first_name|last_name|gender| hire_date|
+------+----------+-----------+---------+------+----------+
| 13001|1957-06-29|     Rosita|  Wiegley|     M|1985-03-05|
| 13002|1954-12-08|       Henk| Muhlberg|     M|1989-05-02|
| 13003|1963-09-21|      Jinxi|     Luft|     M|1991-09-06|
| 13004|1952-12-13|        Fan| Beznosov|     M|1990-05-18|
| 13005|1959-12-26|    Gilbert|   Kobara|     F|1986-12-08|
| 13006|1964-09-01|     Jolita|    Spelt|     M|1986-11-07|
| 13007|1960-08-01|      Yagil|  Frolund|     F|1987-10-09|
| 13008|1955-12-23|     Jayesh|   Zaumen|     M|1996-03-06|
| 13009|1957-11-26|       Marc|    Denis|     F|1995-03-02|
| 13010|1960-12-19|     Sachin|     Bach|     M|1988-08-05|
| 13011|1955-02-25|      Aamer|Glowinski|     F|1989-10-08|
| 13012|1952-02-12|    Florina| Harbusch|     M|1987-12-24|
| 13013|1952-10-17|    Heekeun|     Baak|     F|1993-02-24|
| 13014|1962-06-01|   Gianluca| Budinsky

In [47]:
#Leemos una tabla

ecobici_full = hiveContext.table("spark_data.ecobici").where("genero_usuario='M'")



In [48]:
# Generamos una funcion

def GuardaMiTabla(df,BD,TABLA):
    df2= df.withColumn("Area", lit("PLD"))
    df2.write.mode("overwrite").saveAsTable(BD + "." + TABLA)
    print("La tabla: " + BD + "." + TABLA + " Ha sido generada")


In [49]:
# Guardamos tabla utilizando una funcion

GuardaMiTabla(ecobici_full,"citibanamex","ecobici_mujeres")

La tabla: citibanamex.ecobici_mujeres Ha sido generada


In [109]:
import json

archivoPar="/data_lake/Spark_Citi/input/parametros_citi.json"
parametros  = hiveContext.read.format("json").load(archivoPar, multiLine=True)
parametros.count()

16

In [71]:
data = parametros.rdd.map(lambda x:(x[0],x[1],X[2],x[3])).collectAsMap()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 40.0 failed 4 times, most recent failure: Lost task 1.3 in stage 40.0 (TID 90, quickstart.cloudera, executor 2): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/var/lib/hadoop-yarn/cache/yarn/nm-local-dir/usercache/cloudera/appcache/application_1610581254094_0001/container_1610581254094_0001_01_000003/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/var/lib/hadoop-yarn/cache/yarn/nm-local-dir/usercache/cloudera/appcache/application_1610581254094_0001/container_1610581254094_0001_01_000003/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/var/lib/hadoop-yarn/cache/yarn/nm-local-dir/usercache/cloudera/appcache/application_1610581254094_0001/container_161058125409

In [112]:

fileName = "/data_lake/Spark_Citi/input/parametros_citi.json" 
data  = hiveContext.read.format("json").load(fileName,multiLine=True)
data2 = data.rdd.map(lambda x: x)
data2.collect()



[Row(_corrupt_record=u'['),
 Row(_corrupt_record=u'    {"Hombres":['),
 Row(_corrupt_record=u'              {'),
 Row(_corrupt_record=u'\t\t\t   "BD":"citibanamex",'),
 Row(_corrupt_record=u'\t\t\t   "TABLA":"citi_hombres",'),
 Row(_corrupt_record=u'\t\t\t   "MODO":"overwrite"'),
 Row(_corrupt_record=u'\t\t\t   }'),
 Row(_corrupt_record=u'\t\t\t\t]},'),
 Row(_corrupt_record=u'    {"Mujeres":['),
 Row(_corrupt_record=u'              {'),
 Row(_corrupt_record=u'\t\t\t   "BD":"citibanamex",'),
 Row(_corrupt_record=u'\t\t\t   "TABLA":"citi_mujeres",'),
 Row(_corrupt_record=u'\t\t\t   "MODO":"overwrite"'),
 Row(_corrupt_record=u'\t\t\t   }'),
 Row(_corrupt_record=u'\t\t\t\t]}'),
 Row(_corrupt_record=u']')]